# BYOA Tutorial - Prophet Forecasting en Sagemaker

The following notebook shows how to integrate your own algorithms to Amazon Sagemaker. We are going to go the way of putting together an inference pipeline on the Prophet algorithm for time series. The algorithm is installed in a docker container and then it helps us to train the model and make inferences on an endpoint.


## Step 1: Assemble the dataset

We are going to work with a public dataset that we must download from Kaggle. This dataset is called: Avocado Prices: Historical data on avocado prices and sales volume in multiple US markets and can be downloaded from: https://www.kaggle.com/neuromusic/avocado-prices/download Once downloaded, we must upload it to the same directory where we are running this notebook. The following code prepares the dataset so that Prophet can understand it:

In [1]:
import pandas as pd

# We are left with only the date and the sales
df = pd.read_csv('avocado.csv')
df = df[['Date', 'AveragePrice']].dropna()

df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

# We leave 1 single record per day with the average sales
daily_df = df.resample('D').mean()
d_df = daily_df.reset_index().dropna()

# We format the column names as Prophet expects them
d_df = d_df[['Date', 'AveragePrice']]
d_df.columns = ['ds', 'y']
d_df.head()

# We save the resulting dataset as avocado_daily.csv
d_df.to_csv("avocado_daily.csv",index = False , columns = ['ds', 'y'] )

# Step 2: Package and Upload the Algorithm for Use with Amazon SageMaker


### An overview of Docker

Docker provides a simple way to package code into an _image_ that is completely self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is the same as running a program on the machine, except that the container creates a completely self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you configure the program is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it understands your entire operating environment, including startup commands, environment variables, etc.

In some ways, a Docker container is like a virtual machine, but it is much lighter. For example, a program that runs in one container can start in less than a second, and many containers can run on the same physical machine or virtual machine instance.

Docker uses a simple file called `Dockerfile` to specify how the image is assembled.
Amazon SagMaker uses Docker to allow users to train and implement algorithms.

In Amazon SageMaker, Docker containers are invoked in a certain way for training and in a slightly different way for hosting. The following sections describe how to create containers for the SageMaker environment.


### How Amazon SageMaker runs the Docker container

Because it can run the same image in training or hosting, Amazon SageMaker runs the container with the `train` or` serve` argument. How your container processes this argument depends on the container:

* In the example here, we did not define an ʻENTRYPOINT ʻin the Dockerfile for Docker to execute the `train` command at training time and` serve` at service time. In this example, we define them as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as "ENTRYPOINT" in the Dockerfile, that program will run at startup and its first argument will be either `train` or` serve`. The program can then examine that argument and decide what to do.
* If you are building separate containers for training and hosting (or building just for one or the other), you can define a program as "ENTRYPOINT" in the Dockerfile and ignore (or check) the first argument passed.

#### Run container during training

When Amazon SageMaker runs the training, the `train` script runs like a regular Python program. A series of files are arranged for your use, under the `/ opt / ml` directory:

    /opt/ml
    ├── input
    │   ├── config
    │   │   ├── hyperparameters.json
    │   │   └── resourceConfig.json
    │   └── data
    │       └── <channel_name>
    │           └── <input data>
    ├── model
    │   └── <model files>
    └── output
        └── failure


##### The entrance

* `/ opt / ml / input / config` contains information to control how the program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values ​​will always be strings, so you may need to convert them. `ResourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn does not support distributed training, we will ignore it here.
* `/ opt / ml / input / data / <channel_name> /` (for File mode) contains the input data for that channel. Channels are created based on the call to CreateTrainingJob, but it is generally important that the channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure.
* `/ opt / ml / input / data / <channel_name> _ <epoch_number>` (for Pipe mode) is the pipe for a given epoch. The epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs you can run, but you must close each pipe before reading the next epoch.
    
##### The exit

* `/ opt / ml / model /` is the directory where the model generated by your algorithm is written. Your model can be in any format you want. It can be a single file or an entire directory tree. SagMaker will package any files in this directory into a compressed tar file. This file will be available in the S3 location returned in the `DescribeTrainingJob` output.
* `/ opt / ml / output` is a directory where the algorithm can write a` failure` file that describes why the job failed. The content of this file will be returned in the `FailureReason` field of the` DescribeTrainingJob` result. For successful jobs, there is no reason to write this file as it will be ignored.

#### Running the container during hosting

Hosting has a very different model than training because it must respond to inference requests that arrive through HTTP. In this example, we use recommended Python code to provide a robust and scalable inference request service:

Amazon SagMaker uses two URLs in the container:

* `/ ping` will receive` GET` requests from the infrastructure. Returns 200 if the container is open and accepting requests.
* `/ invocations` is the endpoint that receives inference` POST` requests from the client. The format of the request and the response depends on the algorithm. If the client supplied the `ContentType` and ʻccept` headers, these will also be passed.

The container will have the model files in the same place where they were written during training:

    /opt/ml
    └── model
        └── <model files>


### Container Parts

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageManager:

    .
    ├── Dockerfile
    ├── build_and_push.sh
    └── decision_trees
        ├── nginx.conf
        ├── predictor.py
        ├── serve
        ├── train
        └── wsgi.py


Let's see each one:

* __`Dockerfile`__ describes how to build the Docker container image. More details below.
* __`build_and_push.sh`__ is a script that uses Dockerfile to build its container images and then publishes (push) it to ECR. We will invoke the commands directly later in this notebook, but you can copy and run the script for other algorithms.
* __`prophet`__ is the directory that contains the files to be installed in the container.
* __`local_test`__ is a directory that shows how to test the new container on any machine that can run Docker, including an Amazon SageMaker Notebook Instance. With this method, you can quickly iterate using small data sets to eliminate any structural errors before using the container with Amazon SageMaker.

The files that we are going to put in the container are:

* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as is.
* __`predictor.py`__ is the program that actually implements the Flask web server and Prophet predictions for this application.
* __`serve`__ is the program started when the hosting container starts. It just launches the gunicorn server that runs multiple instances of the Flask application defined in `predictor.py`. You should be able to take this file as is.
* __`train`__ is the program that is invoked when the container for training is executed.
* __`wsgi.py`__ is a small wrapper used to invoke the Flask application. You should be able to take this file as is.

In summary, the two Prophet-specific code files are `train` and` predictor.py`.

### The Dockerfile file

The Dockerfile file describes the image we want to create. It is a description of the complete installation of the operating system of the system you want to run. A running Docker container is significantly lighter than a full operating system, however, because it leverages Linux on the host machine for basic operations.

For this example, we will start from a standard Ubuntu installation and run the normal tools to install the things Prophet needs. Finally, we add the code that implements Prophet to the container and configure the correct environment to run correctly.

The following is the Dockerfile:

In [2]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         curl \
         python-dev \
         build-essential libssl-dev libffi-dev \
         libxml2-dev libxslt1-dev zlib1g-dev \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN curl -fSsL -O https://bootstrap.pypa.io/get-pip.py && \
    python get-pip.py && \
    rm get-pip.py
 
RUN pip --no-cache-dir install \
        numpy \
        scipy \
        sklearn \
        pandas \
        flask \
        gevent \
        gunicorn \
        pystan 

RUN pip --no-cache-dir install \
        fbprophet 
        
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the program in th

### El archivo train

The train file describes the way we are going to do the training.
The Prophet-Docker / container / prophet / train file contains the specific training code for Prophet.
We must modify the train () function in the following way:

    def train():
        print('Starting the training.')
        try:
            # Read in any hyperparameters that the user passed with the training job
            with open(param_path, 'r') as tc:
                trainingParams = json.load(tc)
            # Take the set of files and read them all into a single pandas dataframe
            input_files = [ os.path.join(training_path, file) for file in os.listdir(training_path) ]
            if len(input_files) == 0:
                raise ValueError(('There are no files in {}.\n' +
                                  'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                                  'the data specification in S3 was incorrectly specified or the role specified\n' +
                                  'does not have permission to access the data.').format(training_path, channel_name))
            raw_data = [ pd.read_csv(file, error_bad_lines=False ) for file in input_files ]
            train_data = pd.concat(raw_data)
            train_data.columns = ['ds', 'y']

            # Usamos Prophet para entrenar el modelo.
            clf = Prophet()
            clf = clf.fit(train_data)

            # save the model
            with open(os.path.join(model_path, 'prophet-model.pkl'), 'w') as out:
                pickle.dump(clf, out)
            print('Training complete.')


### El archivo predictor.py

The predictor.py file describes the way we are going to make predictions.
The file Prophet-Docker / container / prophet / predictor.py contains the specific prediction code for Prophet.
We must modify the predict () function in the following way:

    def predict(cls, input):
        """For the input, do the predictions and return them.

        Args:
            input (a pandas dataframe): The data on which to do the predictions. There will be
                one prediction per row in the dataframe"""
        clf = cls.get_model()
        future = clf.make_future_dataframe(periods=int(input.iloc[0]))
        print(int(input.iloc[0]))
        print(input)
        forecast = clf.predict(future)
              
        return forecast.tail(int(input.iloc[0]))



And then the transformation () function as follows:

    def transformation():
        """Do an inference on a single batch of data. In this sample server, we take data as CSV, convert
        it to a pandas data frame for internal use and then convert the predictions back to CSV (which really
        just means one prediction per line, since there's a single column.
        """
        data = None

        # Convert from CSV to pandas
        if flask.request.content_type == 'text/csv':
            data = flask.request.data.decode('utf-8')
            s = StringIO.StringIO(data)
            data = pd.read_csv(s, header=None)
        else:
            return flask.Response(response='This predictor only supports CSV data', status=415, mimetype='text/plain')

        print('Invoked with {} records'.format(data.shape[0]))

        # Do the prediction
        predictions = ScoringService.predict(data)

        # Convert from numpy back to CSV
        out = StringIO.StringIO()
        pd.DataFrame({'results':[predictions]}, index=[0]).to_csv(out, header=False, index=False)
        result = out.getvalue()

        return flask.Response(response=result, status=200, mimetype='text/csv')
 

Basically we modify the line:

        pd.DataFrame({'results':predictions}).to_csv(out, header=False, index=False)
 
By the line:

        pd.DataFrame({'results':[predictions]}, index=[0]).to_csv(out, header=False, index=False)



# Part 3: Using Prophet in Amazon SageMaker
Now that we have all the files created, we are going to use Prophet in Sagemaker

## Container assembly
We start by building and registering the container

In [2]:
%%time
%%sh

# The name of our algorithm
algorithm_name=sagemaker-prophet

cd container

chmod +x prophet/train
chmod +x prophet/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

## Get the login command from ECR and execute it directly
##$(aws ecr get-login --region ${region} --no-include-email)

# Switched to new method of ecr login for docker
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin "${account}.dkr.ecr.${region}.amazonaws.com" 

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# docker build  -t ${algorithm_name} .
# docker tag ${algorithm_name} ${fullname}

# docker push ${fullname}

Login Succeeded
The push refers to repository [373578233960.dkr.ecr.ap-southeast-2.amazonaws.com/sagemaker-prophet]
5f70bf18a086: Preparing
fabe222a751f: Preparing
436e06501221: Preparing
a206157d69db: Preparing
a0078ef23c21: Preparing
555dcabb1b53: Preparing
0a05481263d6: Preparing
2948a9b7d240: Preparing
9edaa71ce233: Preparing
62fdddf6a67c: Preparing
eff16de3ff64: Preparing
61727f5e6796: Preparing
555dcabb1b53: Waiting
0a05481263d6: Waiting
2948a9b7d240: Waiting
9edaa71ce233: Waiting
eff16de3ff64: Waiting
61727f5e6796: Waiting
62fdddf6a67c: Waiting
a206157d69db: Layer already exists
a0078ef23c21: Layer already exists
5f70bf18a086: Layer already exists
436e06501221: Layer already exists
555dcabb1b53: Layer already exists
2948a9b7d240: Layer already exists
0a05481263d6: Layer already exists
9edaa71ce233: Layer already exists
62fdddf6a67c: Layer already exists
eff16de3ff64: Layer already exists
61727f5e6796: Layer already exists
fabe222a751f: Pushed
latest: digest: sha256:ebc759b931399

In [ ]:
## Building the Training Environment
We initialize the session, execution role.

In [7]:
%%sh
pip3 install sagemaker

  Using cached https://files.pythonhosted.org/packages/46/19/c5ab91b1b05cfe63cccd5cfc971db9214c6dd6ced54e33c30d5af1d2bc43/packaging-20.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Stored in directory: /Users/chorder/Library/Caches/pip/wheels/16/0d/24/cbcab58191f1129d3159dabede8d5eb1bf55cfc42fddfb4271
  Stored in directory: /Users/chorder/Library/Caches/pip/wheels/37/42/d8/1609d310cabebc2cf60eca038fa2b0c8503412963734a6fc31
Successfully built sagemaker protobuf3-to-dict


In [1]:
%%time
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

import sagemaker as sage
from time import gmtime, strftime


prefix = 'DEMO-prophet-byo'
# role = get_execution_role()
sess = sage.Session()


CPU times: user 776 ms, sys: 416 ms, total: 1.19 s
Wall time: 2.57 s



# We upload the data to S3

In [2]:
WORK_DIRECTORY = 'data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)
data_location

's3://sagemaker-ap-southeast-2-373578233960/DEMO-prophet-byo'

## We train the model
Using the data uploaded to S3, we train the model by raising an ml.c4.2xlarge instance.
Sagemaker will leave the trained model in the / output directory

In [3]:
%%time

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-prophet:latest'.format(account, region)

tseries = sage.estimator.Estimator(image,
                       'AmazonSageMaker-ExecutionRole-20180723T001212', 
                        1, 
                        'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

tseries.fit(data_location)

2020-11-17 04:30:20 Starting - Starting the training job...
2020-11-17 04:30:23 Starting - Launching requested ML instances......
2020-11-17 04:31:28 Starting - Preparing the instances for training...
2020-11-17 04:32:19 Downloading - Downloading input data
2020-11-17 04:32:19 Training - Downloading the training image......
2020-11-17 04:33:22 Uploading - Uploading generated training model
2020-11-17 04:33:22 Completed - Training job completed
Importing plotly failed. Interactive plots will not work.
Starting the training.
['/opt/ml/input/data/training/avocado_daily.csv']
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Initial log joint probability = -2.69053
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99         450.7    0.00449192       82.0517      0.5589      0.5589      13


## Endpoint assembly for inference
Using the newly trained model, we create an endpoint for inference hosted on an ml.c4.2xlarge instance

In [4]:
%%time

from sagemaker.predictor import csv_serializer
predictor = tseries.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

-------------!CPU times: user 392 ms, sys: 76.7 ms, total: 468 ms
Wall time: 6min 34s


## Inference test
Finally we ask the model to predict the sales for the next 30 days.

In [5]:
%%time
p = predictor.predict("30")
print(p)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
b'"            ds     trend  ...  multiplicative_terms_upper      yhat\n169 2018-03-26  1.473474  ...                         0.0  1.397270\n170 2018-03-27  1.473135  ...                         0.0  1.400523\n171 2018-03-28  1.472796  ...                         0.0  1.403892\n172 2018-03-29  1.472458  ...                         0.0  1.407318\n173 2018-03-30  1.472119  ...                         0.0  1.410742\n174 2018-03-31  1.471781  ...                         0.0  1.414104\n175 2018-04-01  1.471442  ...                         0.0  1.417342\n176 2018-04-02  1.471103  ...                         0.0  1.420400\n177 2018-04-03  1.470765  ...                         0.0  1.423223\n178 2018-04-04  1.470426  ...                         0.0  1.425762\n179 2018-04-05  1.470088  ...                         0.0  1.427975\n180 2018-04-06  1.469749  ...                  